In [46]:
import numpy as np
import h5py
from sklearn import preprocessing
#IMPORT ALL PRE BUILD FUNCTION 
from func import *
# from sklearn.metrics import accuracy_score
import time
from momentum import update_params_with_momentum

In [31]:
#PATH SHOULD BE   YOUR_FOLDER_PATH/popular_optimizer/datasets/train_catvnoncat.h5
hf1 = h5py.File('/pythonfile/popular_optimizer/datasets/train_catvnoncat.h5','r')  #READ THE DATASETS
X_train = hf1.get('train_set_x')     #GET THE X TRAINING DATASETS
Y_train = hf1.get('train_set_y')     #GET Y LABEL 

print("Shape of X_train {}  and  Y_train {}".format(X_train.shape,Y_train.shape))

Shape of X_train (209, 64, 64, 3)  and  Y_train (209,)


In [39]:
#flatten the array and normalize 
X_train = np.reshape(X_train,(X_train.shape[0],-1))  #FLATTEN   (209, 64 *64 * 3)
X_train = preprocessing.normalize(X_train)           #NORMALIZING THE DATASETS

Y_train = np.reshape(Y_train,(Y_train.shape[0],1))                # MAKING SURE IT IS IN CORRECT SHAPE

print("Shape of X_train {}  and  Y_train {}".format(X_train.shape,Y_train.shape))

Shape of X_train (209, 12288)  and  Y_train (209, 1)


In [40]:
def model_simple_gradient_descent(X,Y,learning_rate,num_iter,hidden_size,keep_prob):
    L = len(hidden_size)
    params = initilization(X.shape[1], hidden_size)

    for i in range(1,num_iter):

        cache, A = model_forward(X, params, L,keep_prob)
        cost = cost_f(A, Y)
        grad = backward(X, Y, params, cache, L,keep_prob)
        params = update_params(params, grad, learning_rate, L)
        if i%20 == 0:
            print('cost of iteration________________', i)
            print(cost)
    return params

In [42]:
params = model_simple_gradient_descent(X_train, Y_train, learning_rate = 0.09,
               num_iter=200, hidden_size=[100, 1],keep_prob=0.5)
Y_train_pre = predict(X_train, params, 2)
print('train_accuracy------------', accuracy_score(Y_train_pre, Y_train))

cost of iteration________________ 20
0.6168433550374998
cost of iteration________________ 40
0.6100245970313071
cost of iteration________________ 60
0.5829424468261764
cost of iteration________________ 80
0.5687213202657719
cost of iteration________________ 100
0.5596251854219648
cost of iteration________________ 120
0.5596750289027372
cost of iteration________________ 140
0.5695956625632899
cost of iteration________________ 160
0.5353576834148448
cost of iteration________________ 180
0.5279714359260345
train_accuracy------------ 0.784688995215311


In [47]:
def model_momentum_optimizer(X,Y,learning_rate,num_iter,hidden_size,keep_prob):
    L = len(hidden_size)
    params = initilization(X.shape[1], hidden_size)
    # v = pedding
    for i in range(1,num_iter):

        cache, A = model_forward(X, params, L,keep_prob)
        cost = cost_f(A, Y)
        grad = backward(X, Y, params, cache, L,keep_prob)
        params = update_params_with_momentum(params, grads, v, beta=0.9, learning_rate)
        if i%20 == 0:
            print('cost of iteration________________', i)
            print(cost)
    return params

In [48]:
params = model_momentum_optimizer(X_train, Y_train, learning_rate = 0.09,
               num_iter=200, hidden_size=[100, 1],keep_prob=0.5)
Y_train_pre = predict(X_train, params, 2)
print('train_accuracy------------', accuracy_score(Y_train_pre, Y_train))

TypeError: update_params_with_momentum() missing 1 required positional argument: 'learning_rate'